# LDC Implementation in Python/JAX
Written By: Sungje Park

In [1]:
import jax
from jax import numpy as jnp

import sys
sys.path.append('..')
from src.environment import *
from src.elements import *
from src.dynamics import *
from utils.test_utils import *
from src.methods import *
from tqdm import tqdm
import matplotlib.pyplot as plt

Defining Vars.

In [2]:
rho_ref = 8 # reference density
Mu = .1 # kinematic Viscosity
Re = 10 # reynolds number
N_x = 10 # number of cells in the x dir
N_y = N_x # number of cells in the y dir
dx = 1 # spacing in x
dy = 1 # spacing in y
U_lid = Re*Mu/N_x # velocity of lid
Tau = 3*Mu+.5
dt = 1

In [3]:
cell_mesh = jnp.reshape(jnp.asarray(range(N_x*N_y)),(N_y,N_x))
node_mesh = jnp.reshape(jnp.asarray(range((N_x+1)*(N_y+1))),(N_y+1,N_x+1))
face_mesh = jnp.asarray(range(N_x*(N_y+1)+(N_x+1)*N_y))
vert_mesh = jnp.reshape(face_mesh[0:N_x*(N_y+1)],(N_y,N_x+1))
horz_mesh = jnp.reshape(face_mesh[N_x*(N_y+1):],(N_y+1,N_x))

In [4]:
dynamics = D2Q9(tau=Tau,delta_t = dt)
Element.dynamics = dynamics
Environment.dynamics = dynamics

In [5]:
def create_cell(cell_ind):
    loc = jnp.argwhere(cell_mesh == cell_ind)[0]
    x = loc[1]
    y = loc[0]
    pdf = dynamics.ones_pdf()
    face_ind = jnp.asarray([vert_mesh[y,x],
                            horz_mesh[y,x],
                            vert_mesh[y,x+1],
                            horz_mesh[y+1,x]])
    face_n = jnp.asarray([1,1,-1,-1])
    return Cell.pdf_init(pdf,face_ind,face_n)

cells = [create_cell(ind) for ind in tqdm(range(N_x*N_y))]

100%|██████████| 100/100 [00:02<00:00, 41.64it/s]


In [6]:
def create_node(node_ind):
    loc = jnp.argwhere(node_mesh==node_ind)[0]
    x = loc[1]
    y = loc[0]
    pdf = dynamics.ones_pdf()
    cell_index = []
    cell_dists = []
    cell_dist = jnp.sqrt(2)
    def cell_append(tl,tr,bl,br):
        if tl:
            cell_index.append(cell_mesh[y-1,x-1]) # top left
            cell_dists.append(cell_dist)
        if tr:
            cell_index.append(cell_mesh[y-1,x]) # top right
            cell_dists.append(cell_dist)

        if br:
            cell_index.append(cell_mesh[y,x]) # bottom right
            cell_dists.append(cell_dist)

        if bl:
            cell_index.append(cell_mesh[y,x-1])# bottom left
            cell_dists.append(cell_dist)

    if y == 0:
        type = 1
        vel = jnp.asarray([U_lid,0])
        if x!=0 and x!=N_x:
            cell_append(False,False,True,True)
        elif x==0:
            cell_append(False,False,False,True)
        elif x==N_x:
            cell_append(False,False,True,False)

    elif y == N_y:
        type = 1
        vel = jnp.asarray([0,0])
        if x!=0 and x!=N_x:
            cell_append(True,True,False,False)
        elif x==0:
            cell_append(False,True,False,False)
        elif x==N_x:
            cell_append(True,False,False,False)

    elif x == 0:
        type = 1
        vel = jnp.asarray([0,0])
        # since all corner cases handled
        cell_append(False,True,False,True)

    elif x == N_x:
        type = 1
        vel = jnp.asarray([0,0])
        cell_append(True,False,True,False)
    else:
        type = 0
        cell_append(True,True,True,True)

    node = Node.pdf_init(pdf,jnp.asarray(type),jnp.asarray(cell_index),jnp.asarray(cell_dists))

    if type == 1:
        node.vel = vel
    return node
nodes = [create_node(ind) for ind in tqdm(range((N_x+1)*(N_y+1)))]

100%|██████████| 121/121 [00:02<00:00, 45.95it/s]


In [7]:
node_vel = jnp.asarray([nodes[ind].vel for ind in tqdm(range((N_x+1)*(N_y+1)))])
node_vel = node_vel.reshape(N_y+1,N_x+1,2)


100%|██████████| 121/121 [00:00<00:00, 1390440.50it/s]


In [8]:
def create_face(face_ind):
    pdf = dynamics.ones_pdf()
    nodes_ind = []
    stencil_ind = []
    stencil_dists = jnp.asarray([.5,.5])
    
    if face_ind < N_x*(N_y+1): # vertical faces
        loc = jnp.argwhere(vert_mesh==face_ind)[0]
        x = loc[1]
        y = loc[0]
        nodes_ind.append(jnp.asarray([nodes[node_mesh[y,x]].type,node_mesh[y,x]]))
        nodes_ind.append(jnp.asarray([nodes[node_mesh[y+1,x]].type,node_mesh[y+1,x]]))

        if x == 0:
            stencil_ind.append(-1)
            stencil_ind.append(cell_mesh[y,x])
        elif x == N_x:
            stencil_ind.append(cell_mesh[y,x-1])
            stencil_ind.append(-1)
        else:
            stencil_ind.append(cell_mesh[y,x-1])
            stencil_ind.append(cell_mesh[y,x])

        n = jnp.asarray([1,0])
    else: # horizontal faces
        loc = jnp.argwhere(horz_mesh==face_ind)[0]
        x = loc[1]
        y = loc[0]
        nodes_ind.append(jnp.asarray([nodes[node_mesh[y,x]].type,node_mesh[y,x]]))
        nodes_ind.append(jnp.asarray([nodes[node_mesh[y,x+1]].type,node_mesh[y,x+1]]))

        if y == 0:
            stencil_ind.append(-1)
            stencil_ind.append(cell_mesh[y,x])
        elif y == N_y:
            stencil_ind.append(cell_mesh[y-1,x])
            stencil_ind.append(-1)
        else:
            stencil_ind.append(cell_mesh[y-1,x])
            stencil_ind.append(cell_mesh[y,x])

        n = jnp.asarray([0,1])

    face = Face.pdf_init(pdf,jnp.asarray(nodes_ind),jnp.asarray(stencil_ind),jnp.asarray(stencil_dists),n,jnp.asarray(1))
    return face

faces = [create_face(ind) for ind in tqdm(range((N_x)*(N_x+1)*2))]

100%|██████████| 220/220 [00:03<00:00, 57.78it/s]


In [9]:
env = Environment.create(cells,faces,nodes)
env.setMethods(Methods)
params,config = env.init()

In [10]:
params

{'cells': {'pdf': Array([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.

In [17]:
env.step(params,config)
params

{'cells': {'pdf': Array([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.

In [ ]:
env.step(params,config)
params

Array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [ ]:
env.step(params,config)
params

In [20]:
for i in tqdm(range(10)):
    env.step(params,config)

100%|██████████| 10/10 [00:00<00:00, 5948.52it/s]


In [21]:
params

{'cells': {'pdf': Array([[1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1., 1., 1., 1., 1.